# Introduction to Julia

[Julia](https://julialang.org/) is a fast, dynamic, reprpducible, composable, general and open source programming language that can be downloaded [here](https://julialang.org/downloads/). The full documentation of the current release (v1.7) is available [here](https://docs.julialang.org/en/v1/). This tutorial is designed to go over the basics of Julia, and to cover several topics on which Julia's speed is based. 

1. use/add/remove/develop package
2. basic linear algebra
3. just-in-time (JIT) compiler
4. multiple dispatch

## Package management

In Julia, packages are loaded via `using` (like packages are loaded in python via `import`). We typically load all the packages we want to use at the start of the file as

In [1]:
using LinearAlgebra, Test

To add/remove a package, we can follow [Pkg.jl](https://github.com/JuliaLang/Pkg.jl) and run

In [2]:
using Pkg
Pkg.add("IterativeSolvers")

    Updating registry at `~/.julia/registries/General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
    Updating registry at `~/.julia/registries/SLIMregistryJL`
    Updating git-repo `https://Github.com/slimgroup/SLIMregistryJL.git`
   Resolving package versions...
    Updating `~/.julia/environments/v1.7/Project.toml`
  [42fd0dbc] ~ IterativeSolvers v0.9.2 `https://github.com/JuliaLinearAlgebra/IterativeSolvers.jl.git#master` ⇒ v0.9.2
    Updating `~/.julia/environments/v1.7/Manifest.toml`
  [42fd0dbc] ~ IterativeSolvers v0.9.2 `https://github.com/JuliaLinearAlgebra/IterativeSolvers.jl.git#master` ⇒ v0.9.2
Precompiling project...
  ✓ IterativeSolvers
  ✓ JOLI
  ✓ JUDI
  ✓ MECurvelets
  ✓ InvertibleNetworks
  5 dependencies successfully precompiled in 22 seconds (209 already precompiled)


In [3]:
Pkg.rm("IterativeSolvers")

    Updating `~/.julia/environments/v1.7/Project.toml`
  [42fd0dbc] - IterativeSolvers v0.9.2
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


Note that we can also install a Julia package at a specific version, or from a GitHub repository

In [4]:
Pkg.add(name="IterativeSolvers", version="0.9.2")

   Resolving package versions...
    Updating `~/.julia/environments/v1.7/Project.toml`
  [42fd0dbc] + IterativeSolvers v0.9.2
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


In [5]:
Pkg.add(url="https://github.com/JuliaLinearAlgebra/IterativeSolvers.jl.git", rev="master")

    Updating git-repo `https://github.com/JuliaLinearAlgebra/IterativeSolvers.jl.git`
   Resolving package versions...
    Updating `~/.julia/environments/v1.7/Project.toml`
  [42fd0dbc] ~ IterativeSolvers v0.9.2 ⇒ v0.9.2 `https://github.com/JuliaLinearAlgebra/IterativeSolvers.jl.git#master`
    Updating `~/.julia/environments/v1.7/Manifest.toml`
  [42fd0dbc] ~ IterativeSolvers v0.9.2 ⇒ v0.9.2 `https://github.com/JuliaLinearAlgebra/IterativeSolvers.jl.git#master`
Precompiling project...
  ✓ IterativeSolvers
  ✓ JOLI
  ✓ JUDI
  ✓ MECurvelets
  ✓ InvertibleNetworks
  5 dependencies successfully precompiled in 22 seconds (209 already precompiled)


We can also do these on julia's interactive command-line REPL (read-eval-print loop) via
```julia
] add IterativeSolvers
] rm IterativeSolvers
] add https://github.com/JuliaLinearAlgebra/IterativeSolvers.jl.git
```
etc.

## Linear algebra

The syntax of creating multidimensional arrays is a bit similar to MATLAB. We use square brackets `[]` to set up a multidimensional array, and use semi-colon `;` to switch rows.

In [6]:
A = [1 2; 3 4]

2×2 Matrix{Int64}:
 1  2
 3  4

Note that multidimensional arrays are stored in a column-major order, meaning that they fill up one column at a time. We can verify this by vectorizing `A` via `vec` or `[:]`

In [7]:
A[:]

4-element Vector{Int64}:
 1
 3
 2
 4

In [8]:
vec(A)

4-element Vector{Int64}:
 1
 3
 2
 4

It is also quite simple to create a vector using square bracket, and Julia assumes 1D vector as column vector.

In [9]:
x = [1, 2]

2-element Vector{Int64}:
 1
 2

There are built-in basic linear algebraic operations, such as

In [10]:
inv(A)

2×2 Matrix{Float64}:
 -2.0   1.0
  1.5  -0.5

In [11]:
A * x

2-element Vector{Int64}:
  5
 11

In [12]:
A \ x

2-element Vector{Float64}:
 0.0
 0.5

In [13]:
det(A)

-2.0

## Just-in-time (JIT) compilation

Julia code is just-in-time compiled---i.e., every statement runs using compiled functions which are either compiled right before they are used, or cached compilations from before. This is different from e.g. C, which is compiled before executed. Due to the JIT compilation, the first run of a function/operation typically takes slightly longer time as it needs to compile first. An example is shown below

In [14]:
A = randn(Float32, 10^4, 10^4);
b = randn(Float32, 10^4);

In [15]:
@time A * b;

  0.066466 seconds (290.75 k allocations: 16.182 MiB, 79.50% compilation time)


In [16]:
@time A * b;

  0.013349 seconds (2 allocations: 39.109 KiB)


The first time of execution this "Float32 matrix and Float 32 vector product" takes longer time and more memory allocations as this is the first time to run such functionality in the current notebook.

## Multiple dispatch

The core design to make Julia fast is type-stability through specialization via multiple-dispatch. A single function in Julia can take different types of input and compile the function in specialized ways. This "specialization" brings huge performance gains to Julia. Let's first go over a simple example, namely scalar multiplication, and use the macro `@code_llvm` to check what Julia compiles it to  (LLVM is a type of portable assembly language).

In [17]:
@code_llvm 1 * 2

;  @ int.jl:88 within `*`
define i64 @"julia_*_2642"(i64 signext %0, i64 signext %1) #0 {
top:
  %2 = mul i64 %1, %0
  ret i64 %2
}


In this example, we want to compute the multiplication of 2 integers. As we can see, the underlying assembly code is running the multiplication of integers.

In [18]:
@code_llvm 1.0 * 2.0

;  @ float.jl:405 within `*`
define double @"julia_*_2667"(double %0, double %1) #0 {
top:
  %2 = fmul double %0, %1
  ret double %2
}


If we multiply 2 floating point numbers, we can see that a different (specialized) assembly code is used, which is designed specifically for the type `double` (`Float64` in Julia).

In [19]:
@code_llvm 1.0 * 2

;  @ promotion.jl:380 within `*`
define double @"julia_*_2669"(double %0, i64 signext %1) #0 {
top:
; ┌ @ promotion.jl:350 within `promote`
; │┌ @ promotion.jl:327 within `_promote`
; ││┌ @ number.jl:7 within `convert`
; │││┌ @ float.jl:146 within `Float64`
      %2 = sitofp i64 %1 to double
; └└└└
;  @ promotion.jl:380 within `*` @ float.jl:405
  %3 = fmul double %2, %0
;  @ promotion.jl:380 within `*`
  ret double %3
}


If we want to multiply a floating point number and an integer, Julia promotes the integer first to a floating point number, then performs the multiplication.

This means that the same function (multiplication `*` in this case) is executed via different compiled code blocks that are specialized for different kinds of input. This type-stability feature brings huge performance gain and potentially makes Julia to run almost as fast as C/Fortran (which are strictly typed). With that being said, we should also take advantage of this type-stability when implementing functions---i.e., dispatch a function to multiple types of input.

Let's show a simple example: suppose we want to build a function to take the 2 norm of a scalar/vector/matrix (while we choose to ignore the existing norm functionality). A naive way to implement this would be

In [20]:
function my2norm(x)
    if isa(x, Number)
        return abs(x)
    elseif isa(x, Vector)
        return sqrt(sum(x.^2))
    elseif isa(x, Matrix)
        return return maximum(svdvals(x))
    else
        println("my function is not defined for this kind of input")
        return -1
    end
end

my2norm (generic function with 1 method)

In the above code block, we defined a function `my2norm` that uses nested if-else to find the type of `x` and perform the corresponding operation (absolute value for scalar, euclidean norm for vector, and largest singular value for matrix).

However, there is a smarter and cleaner way to write this function, which provides exactly the same functionality.

In [21]:
mysmart2norm(x::Number) = abs(x)
mysmart2norm(x::Vector) = sqrt(sum(x.^2))
mysmart2norm(x::Matrix) = maximum(svdvals(x))
mysmart2norm(x) = begin println("my function is not defined for this kind of input"); return -1; end

mysmart2norm (generic function with 4 methods)

In [22]:
x = randn(5 * 10^3, 5 * 10^3);
@test my2norm(x) == mysmart2norm(x)

Test Passed
  Expression: my2norm(x) == mysmart2norm(x)
   Evaluated: 141.3436048588063 == 141.3436048588063

The take-away message is: we should be careful of the type-stability aspect of Julia. We should take advantage of the multiple-dispatch feature in julia to write clean and type-stable functions. More performance tips of Julia can be found [here](https://docs.julialang.org/en/v1/manual/performance-tips/).